In [18]:
import numpy as np
import matplotlib.pyplot as plt
import yfinance as yf


In [19]:
ticker = 'CL=F'

data = yf.download(ticker, start='2022-01-01', end='2025-04-07', progress=False)
close_prices = data['Close']

S0 = close_prices.iloc[-1]

S0



Ticker
CL=F    61.990002
Name: 2025-04-04 00:00:00, dtype: float64

In [20]:
returns = close_prices.pct_change().dropna()
sigma = returns.std() * np.sqrt(252)
T = 1
K = 65
num_steps = 252
r = 0.0425
n_sims = 10000
dt = T / num_steps
Z = np.random.normal(size=(n_sims, num_steps))

In [21]:
def simulate_prices(S0, r, sigma, T, num_steps, n_sims):
    dt = T / num_steps
    Z = np.random.normal(size=(n_sims, num_steps)) ## parámetros estándar : 0 y 1
    simulated_returns = (r - 0.5 * sigma.iloc[0]**2) * dt + sigma.iloc[0] * np.sqrt(dt) * Z
    simulared_cumulative_returns = np.cumsum(simulated_returns, axis=1)
    simulated_prices = S0.iloc[0] * np.exp(simulared_cumulative_returns)
    return simulated_prices


def get_price__fixed_strike__average_price(simulated_prices, K, r, T, option_type):
    avg_prices = np.mean(simulated_prices[:, 1:], axis=1)

    if option_type == 'call':
        payoffs = np.maximum(avg_prices - K, 0)
    elif option_type == 'put':
        payoffs = np.maximum(K - avg_prices, 0)

    option_price =  np.exp(-r * T) * np.mean(payoffs)

    return option_price



def get_price__floating_strike__average_price(simulated_prices, St, r, T, option_type):
    avg_prices = np.mean(simulated_prices[:, 1:], axis=1)
    if option_type == 'call':
        payoffs = np.maximum(avg_prices - St, 0)
    elif option_type == 'put':
        payoffs = np.maximum(St - avg_prices, 0)
    option_price =  np.exp(-r * T) * np.mean(payoffs)

    return option_price

def get_price__fixed_strike__geometric_price(simulated_prices, K, r, T, option_type):
    geometric_prices = np.exp(np.mean(np.log(simulated_prices[:, 1:]), axis=1))
    if option_type == 'call':
        payoffs = np.maximum(geometric_prices - K, 0)
    elif option_type == 'put':
        payoffs = np.maximum(K - geometric_prices, 0)
    option_price =  np.exp(-r * T) * np.mean(payoffs)

    return option_price

def get_price__floating_strike__geometric_price(simulated_prices, St, r, T, option_type):
    geometric_prices = np.exp(np.mean(np.log(simulated_prices[:, 1:]), axis=1))
    if option_type == 'call':
        payoffs = np.maximum(geometric_prices - St, 0)
    elif option_type == 'put':
        payoffs = np.maximum(St - geometric_prices, 0)
    option_price =  np.exp(-r * T) * np.mean(payoffs)

    return option_price


def get_price__european(simulated_prices, K, r, T, option_type):
    St = simulated_prices[:, -1]
    if option_type == 'call':
        payoffs = np.maximum(St - K, 0)
    elif option_type == 'put':
        payoffs = np.maximum(K - St, 0)
    option_price =  np.exp(-r * T) * np.mean(payoffs)

    return option_price    


In [24]:
simulated_prices = simulate_prices(S0, r, sigma, T, num_steps, n_sims)
St= simulated_prices[:, -1]
get_price__fixed_strike__average_price(simulated_prices, K, r, T, 'call')

4.440908299441257

In [26]:
get_price__floating_strike__average_price(simulated_prices, St, r, T, 'call')


4.563622766813127

In [27]:
get_price__fixed_strike__geometric_price(simulated_prices, K, r, T, 'call')


4.0733698228438335

In [28]:
get_price__floating_strike__geometric_price(simulated_prices, St, r, T, 'call')


4.246626610163626

In [30]:
get_price__european(simulated_prices, K, r, T, 'call')


8.890309914610295